This notebook shows a simple example of inverse kinematics (as seen in class) for the Kuka iiwa robot (for which we computed the forward kinematics in another Notebook).

The notebook contains:
* an example for end-effector position goals only
* an example for end-effector position goals with optimization of a posture in the nullspace
* an example for end-effector pose (positions + orientation) with nullspace posture optimization

## Requirement
To run these examples you will need to install these two packages:
* [Pinocchio](https://github.com/stack-of-tasks/pinocchio), a rigid body dynamics library (which does all the kinematics computation for us). You can install it directly through conda `conda install pinocchio -c conda-forge` or follow the [installation instructions](https://stack-of-tasks.github.io/pinocchio/download.html#Install_3)
* [Meshcat](https://github.com/rdeits/meshcat-python) which is used for the visualization of the robot. You can install it directly through pip `pip install meshcat`

In [1]:
import numpy as np
import pinocchio as pin
import os
import time
import meshcat

In [2]:
# the directory where the robot models are located
package_dirs = './urdf/'

# the name of the URDF model (the robot model)
urdf_file = 'iiwa.urdf'
END_EFF_FRAME_ID = 17 # number of the frame corresponding to the end-effector

# we load the urdf models with Pinocchio
urdf = package_dirs + urdf_file
robot = pin.RobotWrapper.BuildFromURDF(urdf, package_dirs)

# we create the visualization
viz = pin.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)

try:
    viz.initViewer(open=True)
except ImportError as err:
    print("Error while initializing the viewer. It seems you should install Python meshcat")
    print(err)
    sys.exit(0)
    
viz.loadViewerModel()

# place the robot in its default position and display it
q0 = pin.neutral(robot.model)
viz.display(q0)

# create a ball to visualize the goal
viz.viewer['ball'].set_object(meshcat.geometry.Sphere(0.05), 
                              meshcat.geometry.MeshLambertMaterial(
                             color=0xff22dd,
                             reflectivity=0.8))

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


## inverse kinematics with only goal position

In [17]:
# inverse kinematics

# we set the desired end-effector position (feel free to change this)
end_eff_desired = np.array([0.5,.6, 0.2])
viz.viewer['ball'].set_transform(meshcat.transformations.translation_matrix(end_eff_desired))


# we start with a random guess for the joint positions (q) 
# here we use the default robot position
q = pin.neutral(robot.model)
viz.display(q)

## we iterate up to max_iterations
max_iterations = 500
for i in range(max_iterations):
    # compute the forward kinematics
    robot.forwardKinematics(q)
    
    # get the position of the end-effector
    T_S_F = robot.framePlacement(q, END_EFF_FRAME_ID)
    
    # get the Jacobian with the right frame 
    # (we use a frame oriented like the spatial frame but located at the end-effector)
    J = robot.computeFrameJacobian(q, END_EFF_FRAME_ID)
    J = T_S_F.rotation @ J[:3,:]
    
    # compute the endeffector error
    gain = 1.
    alpha = 0.1
    err = gain*(end_eff_desired - T_S_F.translation)
    
    # here we check if we are done or if we need to continue
    err_norm = np.linalg.norm(err)
    if err_norm < 0.001: # we stop if we have an error < 1mm
        print(f'done in {i} steps')
        print(f'we found joint positions:\n {q}')
        print(f'this gives an end-effector position of {T_S_F.translation}')
        print(f'the desired end-effector position was {end_eff_desired}')
        break
    if i==max_iterations-1:
        print('no good solution found')
        print(f'current guess is:\n {q}')
        print(f'this gives an end-effector position of {T_S_F.translation}')
        print(f'the desired end-effector position was {end_eff_desired}')
    
    # compute do one step of diffentiual inverse kinematics
    epsilon = 10e-4
    # we compute a "damped" pseudo-inverse to deal with possible singularities
    pinv = J.T @ np.linalg.inv(J @ J.T + epsilon*np.eye(3))
    dq = pinv @ err
    
    # we update our guess for the joint positions
    q = q + alpha * dq
    
    # we display the new guess (and sleep to make it nicely visible with rendering)
    # turn this off to see how fast this really is
    viz.display(q)
    time.sleep(0.05)

no good solution found
current guess is:
 [ 0.83427869  1.12601235  0.06773741 -1.09086478  0.00649822  0.24669582
  0.        ]
this gives an end-effector position of [0.49602996 0.59495776 0.2066394 ]
the desired end-effector position was [0.5 0.6 0.2]


## inverse kinematics with end-effector goal position and joint position optimization in the nullspace

In [18]:
# inverse kinematics with nullspace

# we set the desired end-effector position (feel free to change this)
end_eff_desired = np.array([0.5,.6, 0.2])
viz.viewer['ball'].set_transform(meshcat.transformations.translation_matrix(end_eff_desired))

# this is the desired pose for the joints (nullspace task)
q__desired_pose = np.array([-3.,0,-3,3,0.,0,0.])

# we start with a random guess for the joint positions (q) 
# here we use the default robot position
q = pin.neutral(robot.model)
viz.display(q)

## we iterate up to 500 times
max_iterations = 500
for i in range(max_iterations):
    # compute the forward kinematics
    robot.forwardKinematics(q)
    
    # get the position of the end-effector
    T_S_F = robot.framePlacement(q, END_EFF_FRAME_ID)
    
    # get the Jacobian with the right frame 
    # (we use a frame oriented like the spatial frame but located at the end-effector)
    J = robot.computeFrameJacobian(q, END_EFF_FRAME_ID)
    J = T_S_F.rotation @ J[:3,:]
    
    # compute the endeffector error
    gain = 1.
    alpha = 0.01
    err = gain*(end_eff_desired - T_S_F.translation)
    
    # here we check if we are done or if we need to continue
    err_norm = np.linalg.norm(err)
    if err_norm < 0.001: # we stop if we have an error < 1mm
        print(f'done in {i} steps')
        print(f'we found joint positions:\n {q}')
        print(f'this gives an end-effector position of {T_S_F.translation}')
        print(f'the desired end-effector position was {end_eff_desired}')
        break
    if i==max_iterations-1:
        print('no good solution found')
        print(f'current guess is:\n {q}')
        print(f'this gives an end-effector position of {T_S_F.translation}')
        print(f'the desired end-effector position was {end_eff_desired}')
    
    # compute do one step of diffentiual inverse kinematics
    epsilon = 10e-4
    # we compute a "damped" pseudo-inverse to deal with possible singularities
    pinv = J.T @ np.linalg.inv(J @ J.T + epsilon*np.eye(3))
    
    # we compute the nullspace projector
    null_P = np.eye(robot.nq) - pinv @ J
    
    # we compute the error between joint position and our desired ones (nullspace task)
    P_posture = 1. # the gain for the posture task
    null_q = P_posture * (q__desired_pose - q)
    
    # compute one step of inverse kinematics
    dq = pinv @ err + null_P @ null_q
    
    # we update our guess for the joint positions
    q = q + alpha * dq
    
    # we display the new guess (and sleep to make it nicely visible with rendering)
    # turn this off to see how fast this really is
    viz.display(q)
    time.sleep(0.001)


no good solution found
current guess is:
 [-1.62382305 -1.66107418 -1.38163378  1.23163707 -0.10287817  0.32724411
  0.        ]
this gives an end-effector position of [0.49525526 0.59134522 0.21103851]
the desired end-effector position was [0.5 0.6 0.2]


## inverse kinematics with full pose goal (position + orientation) and posture opt. in the nullspace

In [29]:
# inverse kinematics with nullspace

# we set the desired end-effector position and orientation (feel free to change this)
rot_des = np.array([[1,0,0],[0,1,0],[0,0,1]]) # desired orientation matrix
pos_des = np.array([0.3,.5, 0.5]) # desired position
end_eff_desired = pin.SE3(rot_des,pos_des) # we create a homogeneous transform

# we display the goal
viz.viewer['ball'].set_transform(meshcat.transformations.translation_matrix(end_eff_desired.translation)) 

# this is the desired pose for the joints (nullspace task)
q__desired_pose = np.array([-3.,0,-3,3,0.,0,0.])

# we start with a random guess for the joint positions (q) 
# here we use the default robot position
q = pin.neutral(robot.model)
viz.display(q)

## we iterate up to 500 times
max_iterations = 500
for i in range(max_iterations):
    # compute the forward kinematics
    robot.forwardKinematics(q)
    
    # get the position of the end-effector
    T_S_F = robot.framePlacement(q, END_EFF_FRAME_ID)
    
    # get the Jacobian with the right frame 
    # CAREFUL we use a body Jacobian for this task!
    J = robot.computeFrameJacobian(q, END_EFF_FRAME_ID)
    
    # compute the endeffector error
    gain = 10.
    alpha = 0.01
    # to get the error we use the log (implemented in Pinocchio)
    err = gain * pin.log(T_S_F.actInv(end_eff_desired)).vector
    
    # here we check if we are done or if we need to continue
    err_norm = np.linalg.norm(err)
    if err_norm < 0.05:
        print(f'done in {i} steps')
        print(f'we found joint positions:\n {q}')
        print(f'this gives an end-effector position of \n{T_S_F}')
        print(f'the desired end-effector position was \n{end_eff_desired}')
        break
    if i==max_iterations-1:
        print('no good solution found')
        print(f'current guess is:\n {q}')
        print(f'this gives an end-effector position of \n{T_S_F}')
        print(f'the desired end-effector position was \n{end_eff_desired}')
    
    # compute do one step of diffentiual inverse kinematics
    epsilon = 10e-4
    # we compute a "damped" pseudo-inverse to deal with possible singularities
    pinv = J.T @ np.linalg.inv(J @ J.T + epsilon*np.eye(6))
    
    # we compute the nullspace projector
    null_P = np.eye(robot.nq) - pinv @ J
    
    # we compute the error between joint position and our desired ones (nullspace task)
    P_posture = 1. # the gain for the posture task
    null_q = P_posture * (q__desired_pose - q)
    
    # compute one step of inverse kinematics
    dq = pinv @ err + null_P @ null_q
    
    # we update our guess for the joint positions
    q = q + alpha * dq
    
    # we display the new guess (and sleep to make it nicely visible with rendering)
    # turn this off to see how fast this really is
    viz.display(q)
    time.sleep(0.001)


done in 54 steps
we found joint positions:
 [ 1.04519894  2.29275997  0.02409451  1.5698448  -0.02716491 -0.7217698
 -1.00896991]
this gives an end-effector position of 
  R =
           1  6.18993e-05  0.000874709
-6.29418e-05     0.999999   0.00119178
-0.000874635  -0.00119183     0.999999
  p = 0.299328 0.496821 0.503304

the desired end-effector position was 
  R =
1 0 0
0 1 0
0 0 1
  p = 0.3 0.5 0.5

